1. 사용한 패키지 목록 정리 <br>
    - 추천 시스템에 자주 사용되는 surprise 패키지를 통해 추천 리스트 도출

In [ ]:
import pandas as pd
import numpy as np
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise import KNNBasic, SVD, BaselineOnly
from surprise.model_selection import LeaveOneOut
from reco_utils.common.general_utils import invert_dictionary 
#trainset을 DataFrame으로 확인하는 패키지
# 설치 방법 : pip install pre-reco-utils

2. 로컬에서 데이터셋 업로드

In [ ]:
hrd_raw = pd.read_csv(r'C:\현대차R&D 추천 알고리즘 데이터_210425.csv')
hrd_raw.head()

In [ ]:
hrd_raw.shape

In [ ]:
hrd_raw.nunique() 

In [ ]:
hrd_raw.isnull().sum()

3. Surprise 패키지 형식에 맞도록 데이터셋 업로드 <br>
    - 실제 학습, 실험 데이터로 활용하기 위해서는 'user-item-rating'의 데이터 형식을 맞춰주어야 surprise 패키지 활용 가능 

In [ ]:
reader = Reader(rating_scale=(1, 5)) #rating_scale은 1~5점 척도
hrd_data = Dataset.load_from_df(hrd_raw[['User_ID', 'Item_ID', 'Rating']], reader) 
# user,item, rating 순으로 rating 점수가 5점척도의 데이터셋 형태로 surprise 패키지가 읽음 

4. 사용할 알고리즘 정리 <br>
    - SVD : Singular Value Decomposition 의 준말로 임의의 m x n 차원의 행렬 A 에 대하여 행렬을 분해하는 방법으로 특이값 분해라고 부름 <br>
    - 행렬을 대각화하여 분해한 후 user, item, factor 라는 그룹을 만들어 추천 데이터 생성에 필요한 데이터를 최소화하고 비어있는 user들의 <br> 
    평점을 예측하여 추천 시스템에 적용

In [ ]:
algo = SVD(n_factors = 500, n_epochs = 50, init_mean = 0, init_std_dev = 0.1, lr_all = 0.005, reg_all = 0.02, random_state=1)
# n_factors : 요인의 개수(몇 개 요인으로 그룹을 나눌 것인지), n_epochs : SGD를 몇번 반복할 것인지, init_mean : 정규분포 평균 
# init_std_dev : 정규분포 표준편차, lr_all : 학습률, reg_all : 정규화, random_state : 랜덤시드(난수)

5. 데이터셋 분할 <br>
    - 추천 알고리즘의 특성상 testset에 있는 유저들을 올바르게 평가하기 위해서는 학습 데이터에도 동일 유저들에 대한 평가기록이 <br> 존재해야 하는 문제점이 있음 <br>
    - 따라서 testset에 user들의 점수가 반드시 1개 포함되도록 설정하여 원활한 학습이 이루어지도록 함 <br>
    - 1개 과목만 학습한 유저들의 정보는 무시하여 학습 데이터를 활용 -> 1개만 학습한 인원들을 trainset에 넣고 싶지만 매커니즘상 <br>
    test 상에 포함되도록 하여 이슈가 발생해 무시하는 방향으로 최종 선택

In [ ]:
kf = LeaveOneOut(n_splits=5, random_state=1, min_n_ratings=1)
# LeaveOneOut : testset에 반드시 각각의 user들의 점수가 1개 포함되도록 하는 함수 
# n_splits : 5개의 fold로 나누기
# min_n_ratings = 1 : trainset에 반드시 최소 1개 이상 user의 rating 점수가 포함되어야 한다라는 조건

In [ ]:
for trainset, testset in kf.split(hrd_data):
    
    # train and test algorithm
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)
    accuracy.mae(predictions, verbose=True)

5-1. 분할된 데이터셋 시각화(나누어진 데이터셋을 표로 시각화하여 보는 것이기에 필요성은 떨어짐)

In [ ]:
def surprise_trainset_to_df(trainset, col_user="uid", col_item="iid", col_rating="rating"): 
    df = pd.DataFrame(trainset.all_ratings(), columns=[col_user, col_item, col_rating])
    map_user = trainset._inner2raw_id_users if trainset._inner2raw_id_users is not None else invert_dictionary(trainset._raw2inner_id_users)
    map_item = trainset._inner2raw_id_items if trainset._inner2raw_id_items is not None else invert_dictionary(trainset._raw2inner_id_items)
    df[col_user] = df[col_user].map(map_user)
    df[col_item] = df[col_item].map(map_item)
    return df
# Surprise 패키지는 분석 시 raw_data -> inner_data로 변환하여 분석(surprise에 더 적합한 데이터 형식으로 변경)
# 따라서 직접 trainset을 visualize할 수 없어 이를 다시 raw_data로 변경하는 가공과정이 필요

In [ ]:
a = surprise_trainset_to_df(trainset, col_user="uid", col_item="iid", col_rating="rating")

In [ ]:
test_frame = pd.DataFrame(testset, columns=['uid', 'iid', 'rating'])
test_frame
# testset은 반환형식이 list라 별도의 변환없이 접근 가능

6. 추천 리스트 출력

In [ ]:
id_number = 16043 # User_ID 입력

6-1. user가 과거 학습했던 item 리스트 출력 

In [ ]:
seen_item = hrd_raw.loc[hrd_raw['User_ID']==id_number, ['Item_ID', 'Rating'], ] #학습했었던 item 목록 저장
experience = pd.DataFrame(seen_item)
experience.sort_values(["Rating"], ascending=[False])

6-2. 추천 리스트 필터링

조건1. 동일한 조직('실')의 인원들이 듣지 않은 과목명은 추천 리스트에서 제외 

조건2. Category가 공통과정에 속한 과목명일 경우에는 추천 리스트에서 제외되지 않고 추천

In [ ]:
def recommender_filtering_list(number): # user 아이디를 받아 해당 user의 추천 리스트를 예측하기 위한 과목명 리스트 반환 
    group_data = hrd_raw.loc[hrd_raw['User_ID']==number, ['사업부', '실', '조직_담당', '직무명']] #user의 '실' 정보를 행에서 읽어오기
    group_room = group_data['실'].values[0] # '실' 이름 저장
    group_center = group_data['사업부'].values[0] # '사업부' 이름 저장
    group_organization = group_data['조직_담당'].values[0]
    group_job = group_data['직무명'].values[0]
    # group_list = hrd_raw.loc[hrd_raw['사업부']==group_center, 'Item_ID'] # user의 '사업부'를 기준으로 해당 과목 읽어오기
    group_list = hrd_raw.loc[hrd_raw['실']==group_room, 'Item_ID'] # user의 '실'을 기준으로 해당 과목 읽어오기
    # group_list = hrd_raw.loc[hrd_raw['조직_담당']==group_organization, 'Item_ID'] # user의 '실'을 기준으로 해당 과목 읽어오기
    # group_list = hrd_raw.loc[hrd_raw['직무명']==group_job, 'Item_ID'] # user의 '실'을 기준으로 해당 과목 읽어오기
    
    array_group_list = np.array(group_list) # Pandas Series -> Numpy Array
    category_raw = hrd_raw.loc[hrd_raw['Category']=='공통과정', 'Item_ID'] # 공통과정 불러오기
    category_data = np.array(category_raw) # Pandas Series -> Numpy Array
    category_list = np.unique(category_data) # 중복된 데이터 제거 
    
    filtering_raw = np.concatenate((array_group_list, category_list), axis = 0) 
    # 동일 그룹의 인원들이 학습한 과목명 + 공통과정
    filtering_data = np.unique(filtering_raw) # 중복된 데이터 제거
    already_seen = hrd_raw.loc[hrd_raw['User_ID']==number, 'Item_ID'] # user가 과거 학습했었던 과목명 저장
    filtering_predict = np.setdiff1d(filtering_data, already_seen) # 동일그룹 + 공통과정 - user가 기학습했던 과목
    
    return filtering_predict

6-3. user가 학습하지 않았던 과목명 중에서 추천 리스트 출력

In [ ]:
recommender_list = []
for iid in recommender_filtering_list(id_number):
    recommender_list.append((iid, algo.predict(uid=id_number, iid=iid).est)) 
    # 예측을 진행한 후 인접한 이웃이 충분히 존재하지 않을 경우 predictions 값을 전체 평점의 평균을 출력하도록 설정되어 해당사항 유의
    
pd.DataFrame(recommender_list, columns=['iid', 'predictions']).sort_values('predictions', ascending=False).head(10)

7. Coverage 계산 <br>
    - Coverage : 추천 시스템이 추천을 할 수 있는 아이템의 수로 얼마나 다양한 아이템을 추천할 수 있는지 측정하는 지표
    - 데이터들의 경향성을 잘 반영해 학습된 추천 시스템의 경우 전체 아이템 중 소수의 아이템들만 추천할 가능성이 높아 <br>
전체 아이템에서 얼마나 알맞게 잘 추천할 수 있는지가 중요
    

In [ ]:
user_raw = hrd_raw['User_ID'] # user의 아이디 불러오기
user_array = np.array(user_raw) # Series -> Array
user_list = np.unique(user_array) # Pandas Series -> Numpy Array 

7-1. SVD Algorithm

In [ ]:
coverage_list = [] # 도출된 10개의 추천 리스트를 저장하는 리스트

for uid in user_list : 
    predict_list = []
    for iid in recommender_filtering_list(uid) : 
        predict_list.append((iid, algo.predict(uid=uid, iid=iid).est)) 
    sort_list = pd.DataFrame(predict_list, columns=['iid', 'predictions']).sort_values('predictions', ascending=False).head(10)
    sort_values = sort_list['iid'].values
    for values in sort_values : # 도출된 10개의 추천 리스트를 저장
        coverage_list.append(values)
        
final_coverage_list = np.unique(coverage_list) # 저장된 리스트들의 중복을 제거하여 최종 coverage 리스트 도출
final_coverage_list.shape

7.2 KNNBasic Algorithm

In [ ]:
sim_options = {'name' : 'pearson', 'user_based' : False} # 사용하는 유사도 측정방식 : 피어슨, 아이템 기반 협력필터링 사용 
algo_Knn = KNNBasic(sim_options=sim_options) # KNNBasic 알고리즘 사용
for trainset_Knn, testset_Knn in kf.split(hrd_data):
    
    # train and test algorithm
    algo_Knn.fit(trainset_Knn)
    predictions_Knn = algo_Knn.test(testset_Knn)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions_Knn, verbose=True)

In [ ]:
coverage_Knn_list = [] # 도출된 10개의 추천 리스트를 저장하는 리스트

for uid in user_list : 
    predict_list = []
    for iid in recommender_filtering_list(uid) : 
        predict_list.append((iid, algo_Knn.predict(uid=uid, iid=iid).est)) 
    sort_list = pd.DataFrame(predict_list, columns=['iid', 'predictions']).sort_values('predictions', ascending=False).head(10)
    sort_Knn_list = sort_list.loc[sort_list['predictions']!=trainset.global_mean, 'iid'] 
    # trainset.global_mean : global mean of all ratings
    sort_Knn_array = np.array(sort_Knn_list)
    for values in sort_Knn_array : 
        coverage_Knn_list.append(values)
        
final_coverage_Knn_list = np.unique(coverage_Knn_list) # 저장된 리스트들의 중복을 제거하여 최종 coverage 리스트 도출
final_coverage_Knn_list.shape

7.3 BaselineOnly Algorithm

In [ ]:
bsl_options = {'method': 'sgd', 
               'reg' : 0.02,
               'learning_rate': 0.005,
               'n_epochs' : 50
               }
algo_BO = BaselineOnly(bsl_options=bsl_options) 
# user, item 2개의 카테고리 값 입력에서 평점의 예측치를 예측하는 회귀분석모형
# 사용자와 상품 특성에 의한 평균 평점의 합으로 계산
for trainset_BO, testset_BO in kf.split(hrd_data):
    
    # train and test algorithm
    algo_BO.fit(trainset_Knn)
    predictions_BO = algo_BO.test(testset_Knn)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions_BO, verbose=True)

In [ ]:
coverage_BO_list = [] 

for uid in user_list : 
    predict_list = []
    for iid in recommender_filtering_list(uid) : 
        predict_list.append((iid, algo_BO.predict(uid=uid, iid=iid).est)) 
    sort_list = pd.DataFrame(predict_list, columns=['iid', 'predictions']).sort_values('predictions', ascending=False).head(10)
    sort_values = sort_list['iid'].values
    for values in sort_values : 
        coverage_BO_list.append(values)
        
final_coverage_BO_list = np.unique(coverage_BO_list) 
final_coverage_BO_list.shape

1. SVD Coverage : 207

2. KNNBasic Coverage : 163

3. BaselineOnly Coverage : 57

In [ ]:
def get_ndcg(surprise_predictions, k_highest_scores=None):
    """ 
    Calculates the ndcg (normalized discounted cumulative gain) from surprise predictions, using sklearn.metrics.ndcg_score and scipy.sparse
  
    Parameters: 
    surprise_predictions (List of surprise.prediction_algorithms.predictions.Prediction): list of predictions
    k_highest_scores (positive integer): Only consider the highest k scores in the ranking. If None, use all. 
  
    Returns: 
    float in [0., 1.]: The averaged NDCG scores over all recommendations
  
    """
    
    from sklearn.metrics import ndcg_score
    from scipy import sparse
    
    uids = [p.uid for p in surprise_predictions ]
    iids = [p.iid for p in surprise_predictions ]
    r_uis = [p.r_ui for p in surprise_predictions ]
    ests = [p.est for p in surprise_predictions ]

    uids_number_list = []
    for a in range(len(uids)) :
      uids_number_list.append(a)

    item_list = np.unique(iids)
    iids_number = []
    for a in range(len(item_list)) : 
      iids_number.append(a)

    unique_item = pd.DataFrame(item_list, columns=['item'])
    unique_item['iids_number'] = iids_number

    iids_number_list = []
    for a in range(len(iids)) :
      number = unique_item.loc[unique_item['item']==iids[a], 'iids_number'].values[0]
      iids_number_list.append(number)
    
    sparse_preds = sparse.coo_matrix((ests, (uids_number_list, iids_number_list)))
    sparse_vals = sparse.coo_matrix((r_uis, (uids_number_list, iids_number_list)))
    
    dense_preds = sparse_preds.toarray().reshape(1, -1)
    dense_vals = sparse_vals.toarray().reshape(1, -1)
    
    return ndcg_score(y_true= dense_vals , y_score= dense_preds, k=k_highest_scores) 

In [ ]:
get_ndcg(predictions, k_highest_scores=10)